# Malaria Data Cleaning
Malaria is one of the worlds leading causes of death. In this project, I look at the overall numbers of malaria and the trend in estimated cases across different regions between the years 2000 and 2017, focusing on the African continent. I then look at how the number of cases compares with weather conditions, malaria funding and distributed  Insecticide Treated Nets (ITNs). In this notebook, we will take a look at the data, clean it and prepare it for plotting using Pandas. For a visual presentation of the data, please see Malaria_Graphs.ipynb in the same repository.

Data was collected from multiple sources:

World Health Organisation (WHO) (malaria Cases https://www.who.int/data/gho/data/themes/malaria)   
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 
(Funding and ITNs https://www.who.int/publications/i/item/9789241565721)  
World Weather Online  (Weather Conditions www.worldweatheronline.com accessed with API key)  
Berkeley Earth (Global Temperatures http://berkeleyearth.org/archive/data/)  
Global Fund (Funding and ITNs https://data.theglobalfund.org)  
World Bank (Population and Exchange Rates https://data.worldbank.org/)

All the data above is openly available, except the data from *World Weather Online*. See my repository called 'Weather API' for more information on how to access these files.  

First, we read in the necessary libraries.

In [1]:
import pandas as pd
import math

## 1. Malaria Data
The first dataset is from the World Health Organisation (WHO). Two datasets were used:

#### 1) Estimated cases of malaria 
It is hard to predict the true extent of malaria transmission due to a gap in reporting, testing and use of healthcare services. For countries with low malaria transmission, reported numbers were corrected based on local circumstances, such as the extent of healthcare use. In countries with high transmission rates, the number of malaria cases was estimated based on parasite prevalence and population mapping.  

#### 2) Incidence rate of malaria. 
This is the estimated number of malaria cases per 1000 population at risk, with 'population at risk' meaning the population lives in areas where malaria transmission takes place.

Here, I read in the data from WHO, stored as csv files. 

In [2]:
#read in csv files
est_cases_master = pd.read_csv('Data/WHO/cases/malaria_estimated_cases.csv', skiprows=1)
inc_cases =  pd.read_csv('Data/WHO/cases/malaria_case_incidence_per_1000_pop_at_risk.csv', skiprows=1)
#reorder columns
cols = est_cases_master.columns.tolist()
cols = cols[::-1]
cols = cols[-1:] + cols[:-1]
est_cases_master = est_cases_master[cols]
cols_inc = inc_cases.columns.tolist()
cols_inc = cols_inc[::-1]
cols_inc = cols_inc[-1:] + cols_inc[:-1]
inc_cases = inc_cases[cols_inc]

In [3]:
est_cases_master.head()

,Country,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,353 343 [181 000 - 581 000],454 823 [210 000 - 746 000],278 223 [134 000 - 472 000],213 914 [118 000 - 352 000],284 198 [189 000 - 409 000],369 809 [247 000 - 524 000],614 491 [439 000 - 838 000],630 308 [495 000 - 801 000]
1,Algeria,1,1,55,8,0,0,0,0
2,Angola,3 125 901 [2 009 000 - 4 595 000],3 040 461 [1 910 000 - 4 456 000],3 111 760 [1 964 000 - 4 531 000],3 384 997 [2 191 000 - 4 970 000],3 768 087 [2 485 000 - 5 526 000],4 303 582 [2 882 000 - 6 212 000],4 485 050 [3 010 000 - 6 468 000],4 615 605 [3 106 000 - 6 661 000]
3,Argentina,14,0,0,0,0,0,0,0
4,Armenia,0,0,0,0,0,0,0,0


In [4]:
inc_cases.head()

,Country,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,95.54,91.89,104.01,68.41,37.64,27.08,20.53,21.55,17.84,14.77,15.11,18.87,11.15,9.01,12.67,14.91,26.11,27.07,29.01
1,Algeria,0.02,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00
2,Angola,324.44,328.28,312.30,315.35,319.66,323.69,313.81,286.72,241.19,204.35,185.52,175.99,174.43,180.90,187.94,199.99,219.99,228.91,228.91
3,Argentina,2.39,1.15,0.66,0.64,0.60,1.30,1.08,1.95,0.65,0.42,0.07,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Armenia,0.05,0.03,0.02,0.01,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


We can see that the absolute estimated number of cases in the est_cases_master data frame includes the estimated number of cases, along with a minimum and maximum number of cases based on WHO's modelling. However, for countries with low transmission, there are no errors indicated. For the incidence of malaria data, we have just an estimated value of malaria transmission per 1000 population, with no errors attached to that value.

### 1.1 Grouping by World Region

I want to break down the malaria cases by world regions. Different organisations defines its own set of regions and subregions throughout the world. On top of that, the naming system of countries across organisations (and sometimes within the same organisation) is also not consistent. An important step for me when looking at data on countries is to thoroughly crosscheck those names. I created a countries.csv file containing a list of countries and their corresponding world region according to WHO.

Of the 193 official UN member states, 107 countries are included in the WHO dataset. Countries are grouped into 6 WHO regions including Africa, Americas, Southeast Asia, Europe, Eastern Mediterranean, Western Pacific.

In [5]:
countries_WHO_region = pd.read_csv('Data/countries.csv')
countries_WHO_region.head()

,WHO_Region,Country
0,Africa,Algeria
1,Africa,Angola
2,Africa,Benin
3,Africa,Burkina Faso
4,Africa,Cameroon


After crosschecking the names below, I found discrepancies between the WHO malaria data and the WHO list of countries
definition on their website. These were identified and corrected. I also inserted a column in the estimated cases (est_cases_master) data frame containing each country's corresponding WHO region.

In [6]:
#check which countries are mislabelled and are not in study
countries_study=est_cases_master['Country'].tolist()
countries_WHO = countries_WHO_region['Country'].tolist()
for country_study in countries_study:
    if country_study not in countries_WHO:
        print('Mislabelled: {}'.format(country_study))
        for country_WHO in countries_WHO:
            if (country_study[-3:] or country_study[:3]) in country_WHO:
                print('''Could be this name {}
                '''.format(country_WHO))
                
#Assigning WHO Region to each country
country_region=[]
for country in countries_study:
    country_df = countries_WHO_region.loc[countries_WHO_region['Country']==country]
    region_name = country_df.values.tolist()[0][0]
    country_region.append(region_name)
est_cases_master.insert(1,'WHO_Region',country_region,True)

est_cases_master.head()

,Country,WHO_Region,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,Eastern_Mediterranean,353 343 [181 000 - 581 000],454 823 [210 000 - 746 000],278 223 [134 000 - 472 000],213 914 [118 000 - 352 000],284 198 [189 000 - 409 000],369 809 [247 000 - 524 000],614 491 [439 000 - 838 000],630 308 [495 000 - 801 000]
1,Algeria,Africa,1,1,55,8,0,0,0,0
2,Angola,Africa,3 125 901 [2 009 000 - 4 595 000],3 040 461 [1 910 000 - 4 456 000],3 111 760 [1 964 000 - 4 531 000],3 384 997 [2 191 000 - 4 970 000],3 768 087 [2 485 000 - 5 526 000],4 303 582 [2 882 000 - 6 212 000],4 485 050 [3 010 000 - 6 468 000],4 615 605 [3 106 000 - 6 661 000]
3,Argentina,Americas,14,0,0,0,0,0,0,0
4,Armenia,Europe,0,0,0,0,0,0,0,0


Next I split the data into three separate data frames for the estimated malaria cases, the minimum cases and the maximum transmission. For values without errors, I filled the cell with NaN values.

In [7]:
#Creating separate dataframe for median, min, max
est_cases = est_cases_master.copy()    
est_cases_min = est_cases_master.copy()
est_cases_max = est_cases_master.copy()
est_cases_key_list = est_cases.columns.values.tolist()
est_cases_key_list = est_cases_key_list[2:]
for key in est_cases_key_list:
    est_cases_minmax_list=[]
    for num in est_cases_min[key].values:
        if '[' in num:
            est_cases_minmax_list.append(num)
        else:
            est_cases_minmax_list.append('nan')
    est_cases_min[key] = est_cases_minmax_list
    est_cases_max[key] = est_cases_minmax_list
for key in est_cases_key_list:
    est_cases[key] = [num.split('[')[0].replace(' ','') for num in est_cases[key].values]
    est_cases_min[key] = [num.split('[')[-1].split('-')[0].replace(' ','') for num in est_cases_min[key].values]
    est_cases_max[key] = [num.split('-')[-1].replace(']','').replace(' ','') for num in est_cases_max[key].values]
for key in est_cases_key_list:
    est_cases[key] = [int(num) for num in est_cases[key].values]
    est_cases_min[key] = [float(num) for num in est_cases_min[key].values]
    est_cases_max[key] = [float(num) for num in est_cases_max[key].values]

In [8]:
est_cases.head()

,Country,WHO_Region,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,Eastern_Mediterranean,353343,454823,278223,213914,284198,369809,614491,630308
1,Algeria,Africa,1,1,55,8,0,0,0,0
2,Angola,Africa,3125901,3040461,3111760,3384997,3768087,4303582,4485050,4615605
3,Argentina,Americas,14,0,0,0,0,0,0,0
4,Armenia,Europe,0,0,0,0,0,0,0,0


In [9]:
est_cases_max.head()

,Country,WHO_Region,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,Eastern_Mediterranean,581000.0,746000.0,472000.0,352000.0,409000.0,524000.0,838000.0,801000.0
1,Algeria,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,Africa,4595000.0,4456000.0,4531000.0,4970000.0,5526000.0,6212000.0,6468000.0,6661000.0
3,Argentina,Americas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Armenia,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


I wanted to find the prevalence of malaria per region, so I grouped the data frame into WHO regions and summed up the estimated malaria cases per region. I wanted each row to contain case numbers for a specific year, so I melted the data frame and then pivoted it. This format makes it easy and convenient for plotting bar graphs with Plotly.

In [10]:
#Summing estimated cases
WHO_region = est_cases.groupby('WHO_Region').sum().reset_index() 
#Pivot dataframe
WHO_region_melt = WHO_region.melt(id_vars='WHO_Region',
                                   var_name='Year',
                                   value_name='Cases')
WHO_region_pivot = WHO_region_melt.pivot(
        columns='WHO_Region',
        index='Year',
        values='Cases').reset_index()

In [11]:
WHO_region.head()

,WHO_Region,2010,2011,2012,2013,2014,2015,2016,2017
0,Africa,206346611,200953458,201187840,200507228,196177100,193784951,195459072,200457877
1,Americas,811288,609058,579926,563129,481295,572697,711809,975003
2,Eastern_Mediterranean,4255153,4407682,4086100,3824390,4560612,4385319,4512157,4424763
3,Europe,167,69,21,3,2,0,0,0
4,Southeast_Asia,25534650,21532362,18702703,14037687,13500165,13977380,14236170,11287365


In [12]:
WHO_region_pivot.head(10)

WHO_Region,Year,Africa,Americas,Eastern_Mediterranean,Europe,Southeast_Asia,Western_Pacific
0,2010,206346611,811288,4255153,167,25534650,1837856
1,2011,200953458,609058,4407682,69,21532362,1585959
2,2012,201187840,579926,4086100,21,18702703,1887698
3,2013,200507228,563129,3824390,3,14037687,2032567
4,2014,196177100,481295,4560612,2,13500165,2353596
5,2015,193784951,572697,4385319,0,13977380,1450604
6,2016,195459072,711809,4512157,0,14236170,1735557
7,2017,200457877,975003,4424763,0,11287365,1856649


Here, I wanted to compare the average incidence rate of malaria transmission for different regions, by assigning each country a WHO region and then grouping the data frame by that region. I melted the data frame for convenience of plotting.

In [13]:
#Adding WHO region to each country for incidence rate
countries_study_inc=inc_cases['Country'].tolist()
inc_country_region=[]
for country in countries_study_inc:
    inc_country_region.append(countries_WHO_region[countries_WHO_region['Country']==country].values.tolist()[0][0])
years_list_inc = [str(num) for num in range(2000,2019)]
inc_cases.insert(1,'WHO_Region',inc_country_region,True)
#Calculating the mean for each WHO region
inc_cases_region = inc_cases.groupby('WHO_Region').mean().reset_index()
inc_cases_region_ = pd.melt(inc_cases_region, id_vars=['WHO_Region'], value_vars=years_list_inc,
                        var_name='Year', value_name='Inc_Cases')

In [14]:
inc_cases_region.head()

,WHO_Region,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Africa,305.295000,303.958182,290.494091,280.900455,269.802273,252.069773,242.454773,235.520682,229.790909,228.738409,224.817955,216.487273,213.689773,208.956136,199.032727,198.866818,201.032727,197.982273,189.577727
1,Americas,32.759500,34.482000,28.013000,21.882500,16.621500,16.799000,9.376500,6.938500,6.098500,5.656000,6.149000,5.191500,5.011500,5.958000,3.299000,3.159000,3.860000,4.886500,5.510000
2,Eastern_Mediterranean,28.844286,28.055714,26.833571,23.864286,19.373571,20.191429,18.393571,17.521429,11.948571,10.114286,10.741429,9.434286,8.868571,8.212857,9.207143,9.434286,12.062143,12.728571,13.891429
3,Europe,2.875556,2.652222,2.203333,1.640000,1.220000,0.724444,0.334444,0.166667,0.073333,0.053333,0.031111,0.005556,0.002222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Southeast_Asia,30.080000,26.657000,21.381000,16.466000,28.335000,22.282000,20.497000,19.913000,23.051000,19.532000,20.861000,11.054000,10.014000,4.562000,3.611000,2.930000,2.439000,2.184000,1.446000


In [15]:
inc_cases_region_.head(10)

,WHO_Region,Year,Inc_Cases
0,Africa,2000,305.295000
1,Americas,2000,32.759500
2,Eastern_Mediterranean,2000,28.844286
3,Europe,2000,2.875556
4,Southeast_Asia,2000,30.080000
5,Western_Pacific,2000,117.056000
6,Africa,2001,303.958182
7,Americas,2001,34.482000
8,Eastern_Mediterranean,2001,28.055714
9,Europe,2001,2.652222


### 1.2 Grouping by African Subregion and Countries
As the overwhelming majority of malaria cases occurs in Africa, I wanted to take a closer look at the continent and break down the cases per African subregion and country. For this, I used the African Union's (AU) definition of which countries it considers part of Africa and their definition of African subregions. The list is defined in the dictionary below. I crosschecked and corrected the country names between the AU and WHO organisations and selected the AU countries from the WHO data.

In [16]:
AU_Dict={
        'SA':['Angola','Botswana','Eswatini','Malawi','Mozambique',
              'Namibia','South Africa','Zambia','Zimbabwe'],

        'EA':['Comoros','Djibouti','Eritrea','Ethiopia','Kenya','Madagascar',
              'Mauritius','Rwanda','Seychelles','Somalia','South Sudan','Sudan',
              'United Republic of Tanzania','Uganda'],

        'CA':['Burundi','Cameroon','Central African Republic','Chad',
              'Congo','Democratic Republic of Congo',
              'Equatorial Guinea','Gabon','Sao Tome and Principe'],

        'NA':['Algeria','Egypt','Libyan Arab Jamahiriya','Morocco','Tunisia'],

        'WA':['Benin','Burkina Faso','Cape Verde','Côte d\'Ivoire',
              'Gambia','Ghana','Guinea','Guinea-Bissau','Liberia',
              'Mali','Niger','Nigeria','Senegal','Sierra Leone','Togo','Mauritania']
        }

#Cross checking WHO and AU naming system
#Rename mislabelled countries
for region,countries in AU_Dict.items():
    for country in countries:
        if country not in countries_WHO:     
            print('**Mislabelled: {}'.format(country))
            for name in countries_WHO:
                #print(country, name)
                if (country[-3:] or country[:3]) in name:
                    print('Could be this name {}'.format(name))

#Selecting African countries 
AU_country_list=[]
for region,countries in AU_Dict.items():
    for country in countries:
        AU_country_list.append(country)
inc_cases_africa = inc_cases[inc_cases['Country'].isin(AU_country_list)].reset_index(drop=True)
inc_cases_africa = inc_cases_africa.drop('WHO_Region',axis=1)
inc_cases_africa_countrylist = inc_cases_africa['Country'].tolist()

In order to assess the extent of malaria transmission in African subregions, I assigned the AU region to each country as a new column in the data frame containing incidence rates. I found the average number of cases within the region per year by grouping the data frame into AU regions. I melted the data frame for plotting purposes.

In [17]:
#Assiging AU region to each African country
inc_cases_africa_regionlist=[]
for country in inc_cases_africa_countrylist:
    for region,countries in AU_Dict.items():
        if country in countries:
            inc_cases_africa_regionlist.append(region)            
inc_cases_africa.insert(1,'AU_Region',inc_cases_africa_regionlist,True)

inc_cases_africa.head()

,Country,AU_Region,2000,2001,2002,2003,2004,2005,2006,2007,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Algeria,NA,0.02,0.00,0.01,0.00,0.00,0.00,0.00,0.01,...,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00
1,Angola,SA,324.44,328.28,312.30,315.35,319.66,323.69,313.81,286.72,...,204.35,185.52,175.99,174.43,180.90,187.94,199.99,219.99,228.91,228.91
2,Benin,WA,410.94,428.66,435.07,454.31,475.38,487.04,492.73,480.24,...,417.96,387.76,370.11,378.03,395.00,399.24,411.82,421.58,399.56,386.18
3,Botswana,SA,17.88,7.03,3.28,1.63,1.04,1.25,4.01,1.03,...,2.10,1.69,0.51,0.22,0.53,1.50,0.37,0.81,2.05,0.59
4,Burkina Faso,WA,589.33,588.45,580.57,566.27,537.01,506.78,493.78,503.80,...,552.03,551.24,539.56,527.54,487.52,436.06,400.09,401.73,399.94,398.73


In [18]:
#Incidence rate of malaria in AU region
inc_AU_region = inc_cases_africa.groupby('AU_Region').mean().reset_index()
inc_AU_region = pd.melt(inc_AU_region, id_vars=['AU_Region'], value_vars=years_list_inc,
                        var_name='Year', value_name='Inc_Cases')
inc_AU_region.head(10)

,AU_Region,Year,Inc_Cases
0,CA,2000,378.022222
1,EA,2000,181.805833
2,NA,2000,0.006667
3,SA,2000,203.924444
4,WA,2000,389.577500
5,CA,2001,380.723333
6,EA,2001,181.669167
7,NA,2001,0.000000
8,SA,2001,207.166667
9,WA,2001,382.345625


Calculating the difference in estimated, maximum and minimum values and storing the error values in a column. I then sorted the data frame rows based on malaria case numbers in 2017.

In [19]:
#Estimated cases in African countries 
est_cases_african_countries = est_cases[est_cases['WHO_Region']=='Africa'].reset_index(drop=True)
est_cases_african_countries['est_cases_max_2017'] = est_cases_max[est_cases_max['WHO_Region']=='Africa']['2017'].reset_index(drop=True) - est_cases_african_countries['2017']
est_cases_african_countries['est_cases_min_2017'] = est_cases_african_countries['2017'] - est_cases_min[est_cases_min['WHO_Region']=='Africa']['2017'].reset_index(drop=True)
est_cases_african_countries = est_cases_african_countries.sort_values('2017').reset_index(drop=True)

est_cases_african_countries.head()

,Country,WHO_Region,2010,2011,2012,2013,2014,2015,2016,2017,est_cases_max_2017,est_cases_min_2017
0,Algeria,Africa,1,1,55,8,0,0,0,0,NaN,NaN
1,Cape Verde,Africa,47,7,1,22,26,7,48,423,NaN,NaN
2,Eswatini,Africa,268,549,562,962,711,157,350,724,NaN,NaN
3,Sao Tome and Principe,Africa,2740,8442,12550,9243,1754,2058,2238,2239,NaN,NaN
4,Botswana,Africa,3072,678,302,725,2065,519,1150,2989,1211.0,689.0


Below I calculated each countries percentage share of malaria cases and selected the countries with the highest percentage of cases.

In [20]:
#Countries with highest percentage of worldwide cases
total_cases = est_cases['2017'].sum()
est_cases['2017_Perc'] = (est_cases['2017']/total_cases)*100
est_cases_top5=est_cases[est_cases['2017_Perc']>3.0].sort_values('2017_Perc')
est_cases = est_cases.drop('2017_Perc',axis=1)

est_cases_top5.head(15)

,Country,WHO_Region,2010,2011,2012,2013,2014,2015,2016,2017,2017_Perc
59,Mali,Africa,5772983,6279267,6961475,7448756,7468113,6833022,6902717,7160192,3.269469
17,Cameroon,Africa,5361329,5462230,6168031,6960282,7282873,7223417,7176158,7307515,3.336740
68,Niger,Africa,7007707,7323097,7660985,7780901,7700900,7397212,7457829,7702777,3.517223
40,Ghana,Africa,9171294,9251148,9004550,8552123,8113023,7513657,7652909,7805045,3.563921
13,Burkina Faso,Africa,9221846,9320362,9228345,8736057,8258938,7770245,7675183,7907562,3.610732
97,Uganda,Africa,11503116,10686032,9074100,7407982,7040282,8182789,8425793,8600724,3.927242
47,India,Southeast_Asia,20490000,17520000,14220000,11210000,11420000,12200000,12630000,9590000,4.378962
63,Mozambique,Africa,8455521,8930459,9576806,9644363,9425097,9496089,9753050,10025823,4.577967
27,Democratic Republic of Congo,Africa,23691683,22535174,22281939,22488362,23031390,24159871,24454696,25021891,11.425435
69,Nigeria,Africa,60749349,60529456,61587135,62020888,59365039,52697962,52357005,53667565,24.505552


In [21]:
#Incidence rate for African countries
inc_cases_africa=inc_cases_africa.sort_values('2018').reset_index(drop=True)
inc_cases_africa.tail(5)

,Country,AU_Region,2000,2001,2002,2003,2004,2005,2006,2007,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
44,Liberia,WA,478.34,475.18,461.73,446.45,428.70,404.63,388.81,383.18,...,368.81,345.77,330.41,307.90,317.28,337.57,346.97,386.30,401.11,361.50
45,Benin,WA,410.94,428.66,435.07,454.31,475.38,487.04,492.73,480.24,...,417.96,387.76,370.11,378.03,395.00,399.24,411.82,421.58,399.56,386.18
46,Mali,WA,406.23,407.45,416.46,423.81,429.37,431.24,405.71,388.81,...,374.56,383.60,404.73,435.65,452.82,441.01,391.83,384.22,386.78,386.78
47,Burkina Faso,WA,589.33,588.45,580.57,566.27,537.01,506.78,493.78,503.80,...,552.03,551.24,539.56,527.54,487.52,436.06,400.09,401.73,399.94,398.73
48,Rwanda,EA,223.17,213.38,171.70,144.92,126.80,154.61,154.02,90.64,...,158.16,126.31,39.29,71.46,121.45,219.81,341.96,585.54,538.34,486.49


## 2. Weather Data

I wanted to look at the correlation between the number of malaria cases and weather conditions. For this, I used data from *World Weather Online*, which was available from July 2008. I accessed daily averages in temperature, cloud cover, humidity, maximum temperature, minimum temperature, precipitation and pressure. The values are from historical forecast data and are not recorded values. To ensure the forecast data was suitable, I crosschecked the *World Weather Online* temperature data with recorded temperatures from *Berkeley Earth*. 

In [22]:
#Berkeley Earth temperature data (use as cross check)
temp_df = pd.read_csv('Data/Berkeley_Earth/GlobalLandTemperaturesByCountry.csv')
temp_df = temp_df[temp_df['Country'].isin(inc_cases_africa_countrylist)].reset_index(drop=True)
temp_df['Year'] = [num.split('-')[0] for num in temp_df.Date.values]
temp_df= temp_df.groupby(['Country','Year']).mean().reset_index()
list5 = [str(num) for num in range(2000,2014)]
temp_df = temp_df[temp_df['Year'].isin(list5)].reset_index(drop=True)
temp_df_allyears=pd.DataFrame({'Country':[],'Year':[],'AverageTemperature':[],
        'AverageTemperatureUncertainty':[],'Malaria_Cases_Year':[]})
for Selected_Year in list5:
    temp_df_year = temp_df[temp_df['Year']==Selected_Year].reset_index(drop=True)
    temp_df_countries = temp_df_year['Country'].tolist()
    inc_cases_africa_chopped=inc_cases_africa[inc_cases_africa['Country'].isin(temp_df_countries)].reset_index(drop=True)
    temp_df_year=temp_df_year.sort_values('Country').reset_index(drop=True)
    inc_cases_africa_chopped=inc_cases_africa_chopped.sort_values('Country').reset_index(drop=True)
    temp_df_year['Malaria_Cases_Year']=inc_cases_africa_chopped[Selected_Year]
    temp_df_allyears=temp_df_allyears.append(temp_df_year)

Below, I read in the data from *World Weather Online* and selected only the African countries. The data was stored in daily time intervals. The malaria cases were defined on a yearly basis, so I grouped the weather data by country and year. 

In [23]:
#Reading and Organising World Weather Online Data
WWO_Africa_list=['Angola','Botswana','Lesotho','Malawi','Mozambique','Namibia','South_Sudan','Sudan',
             'South_Africa','Zambia','Zimbabwe','Comoros','Djibouti','Eritrea','Ethiopia','Kenya',
             'Madagascar','Tanzania','Uganda','Burundi','Cameroon','Central_African_Republic',
             'Chad','Democratic_Republic_of_Congo','Congo','Equatorial_Guinea','Gabon','Mauritius','Rwanda','Seychelles',
             'Somalia','Sao_Tome_and_Principe','Algeria','Egypt','Libya','Morocco','Tunisia',
             'Benin','Burkina_Faso','Cape_Verde', "Cote_d'Ivoire",'Gambia','Ghana','Guinea',
             'Guinea-Bissau','Liberia','Mali','Niger','Nigeria','Senegal','Sierra_Leone','Togo',
             'Mauritania']
WWO_years = [str(num) for num in range(2009,2018)] 
WWO_df_master = pd.read_csv('Data/World_Weather_Online/countries/2013/Algeria.csv')
WWO_df_columns = WWO_df_master.columns.tolist()
WWO_df_columns = WWO_df_columns[0:3] + WWO_df_columns[16:21] + WWO_df_columns[24:25]
WWO_df = pd.DataFrame()
for year in WWO_years:
    for country in WWO_Africa_list:
        WWO_df_master = pd.read_csv('Data/World_Weather_Online/countries/{}/{}.csv'.format(year,country))
        WWO_df = WWO_df.append(WWO_df_master[WWO_df_columns], ignore_index=True)
WWO_df['Year'] = [date.split('-')[0] for date in WWO_df['date_time'].values]
WWO_df = WWO_df.groupby(['location','Year']).mean().reset_index()
WWO_inc = inc_cases.drop(['WHO_Region'],axis=1)
WWO_inc = pd.melt(inc_cases, id_vars=['Country'], value_vars=WWO_years,
                  var_name='Year', value_name='Inc_Cases')

In [24]:
WWO_df_master.head()

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
0,2017-01-01,30,20,0.0,10.1,6,20,09:55 AM,09:41 PM,07:36 AM,...,38,14,26,0.0,1015,30,10,69,21,Mauritania
1,2017-01-02,30,20,0.0,11.0,6,27,10:39 AM,10:35 PM,07:36 AM,...,32,2,26,0.0,1014,30,10,71,18,Mauritania
2,2017-01-03,31,21,0.0,11.1,7,34,11:23 AM,11:29 PM,07:36 AM,...,36,2,24,0.0,1015,31,10,76,19,Mauritania
3,2017-01-04,30,20,0.0,11.0,6,41,12:05 PM,No moonset,07:37 AM,...,41,7,22,0.0,1017,30,10,67,23,Mauritania
4,2017-01-05,28,18,0.0,11.1,6,49,12:49 PM,12:24 AM,07:37 AM,...,37,14,21,0.0,1016,28,10,65,21,Mauritania


In [25]:
WWO_df.head()

,location,Year,maxtempC,mintempC,cloudcover,humidity,precipMM,pressure,tempC
0,Algeria,2009,21.800000,14.895890,18.873973,66.780822,0.643836,1015.846575,21.800000
1,Algeria,2010,21.057534,14.008219,23.528767,69.126027,0.914521,1014.402740,21.057534
2,Algeria,2011,21.482192,14.263014,22.136986,69.865753,0.885753,1017.364384,21.482192
3,Algeria,2012,21.494536,14.275956,21.021858,67.693989,1.035519,1017.202186,21.494536
4,Algeria,2013,20.747945,13.928767,23.443836,69.961644,1.191781,1016.501370,20.747945


To align the names of countries between datasets, I created two functions to identify discrepancies between names and to rename mismatching countries.

In [26]:
#crosschecking naming systems
def Names_Crosscheck(DF1_col, DF2_col, print_out=True):
    for country in DF1_col.tolist():
        if country not in DF2_col.tolist():
            if print_out==True:
                print(country)
            else:
                pass
def Rename(old_names, new_names, DF):
    for i in range(len(old_names)):
        DF = DF.replace(old_names[i],new_names[i])
    return DF

In [27]:
#cross checking WWO_inc naming system with WWO_df names
WWO_df['WWO_Country'] = [name.replace('_',' ') for name in WWO_df['location'].values]
Names_Crosscheck(WWO_df['WWO_Country'], WWO_inc['Country'], print_out=False)
old_names = ["Cote d'Ivoire",'Lesotho','Tanzania']       
new_names = ["Côte d'Ivoire",'Eswatini','United Republic of Tanzania']
for i in range(3):
    WWO_df['WWO_Country'] = [name.replace(old_names[i],new_names[i]) for name in WWO_df['WWO_Country'].values]
WWO_df = WWO_df.drop(['location'],axis=1)

After crosschecking the country names stored in the two data frames, I combined the incident rate of malaria information with the weather conditions for that year into one data frame.

In [28]:
#merging data frames
WWO_df = pd.merge(WWO_df, WWO_inc, how='inner', left_on=['WWO_Country','Year'], right_on=['Country','Year'])
WWO_df = WWO_df.drop('WWO_Country',axis=1)
WWO_Final_columns = WWO_df.columns.tolist()
WWO_Final_columns = [WWO_Final_columns.pop(8)] + WWO_Final_columns
WWO_df = WWO_df[WWO_Final_columns]

WWO_df.head()

,Country,Year,maxtempC,mintempC,cloudcover,humidity,precipMM,pressure,tempC,Inc_Cases
0,Algeria,2009,21.800000,14.895890,18.873973,66.780822,0.643836,1015.846575,21.800000,0.00
1,Algeria,2010,21.057534,14.008219,23.528767,69.126027,0.914521,1014.402740,21.057534,0.00
2,Algeria,2011,21.482192,14.263014,22.136986,69.865753,0.885753,1017.364384,21.482192,0.00
3,Algeria,2012,21.494536,14.275956,21.021858,67.693989,1.035519,1017.202186,21.494536,0.03
4,Algeria,2013,20.747945,13.928767,23.443836,69.961644,1.191781,1016.501370,20.747945,0.00


## 3. Funding Data

Data on healthcare funding was provided by Global Fund and available for download from WHO and the Global Fund website. To get an overall picture of funding, datasets on foreign healthcare funding, yearly exchange rates, population and distribution of commodities such as Insecticide Treated Nets were used.

### 3.1 Organisation and Government Funding

The data was stored as Excel files and read in below. Unnecessary columns and rows were dropped, columns were renamed, country names were again crosschecked, empty cells were filled with necessary information and values were converted to the correct data type.

In [29]:
#Funding per Capita by Country and Year
Funding_df = pd.read_excel('Data/WHO/funding.xls',sheet_name='Funding')
column_list = ['Unnamed: {}'.format(str(num)) for num in range(7,15)]
Funding_df = Funding_df.drop(column_list,axis=1)
column_list = ['Country', 'Year', 'Global_Fund', 'PMI_USAID', 'World_Bank', 'UK', 'Government (local)']
Funding_df.columns = column_list
Funding_df = Funding_df.drop([0,1,2,3],axis=0)
Funding_df = Funding_df.drop(range(289,302),axis=0)
row_list = ['AFRICAN','SOUTH-EAST ASIA','AMERICAS','EASTERN MEDITERRANEAN','WESTERN PACIFIC']
for i in range(len(row_list)):
    Funding_df = Funding_df.drop(Funding_df[Funding_df['Country']==row_list[i]].index,axis=0)
Funding_df = Funding_df.replace(['United Republic of Tanzania9'],'United Republic of Tanzania')
Funding_df = Funding_df.replace(['South Sudan8'],'South Sudan')
Funding_df = Funding_df.reset_index(drop=True)
Funding_df = Funding_df.drop([279],axis=0)
count=0
for cell in Funding_df['Country'].values:
    if count%3 == 0:
        country = cell
    if count%3 != 0:
        Funding_df.at[count, 'Country'] = country
    count += 1
    
#remove Parag. bc no Government data given
Funding_df = Funding_df.drop([188],axis=0).reset_index(drop=True)
#convert values to float and sum rows
column_list = ['Global_Fund', 'PMI_USAID', 'World_Bank', 'UK', 'Government (local)']
Funding_df[column_list] = Funding_df[column_list].apply(pd.to_numeric)

Funding_df.head()

,Country,Year,Global_Fund,PMI_USAID,World_Bank,UK,Government (local)
0,Algeria,2016.0,0.0,0,0,0,1743483
1,Algeria,2017.0,0.0,0,0,0,1748498
2,Algeria,2018.0,0.0,0,0,0,1812462
3,Angola,2016.0,2725165.0,28133718,0,0,50874556
4,Angola,2017.0,15453275.0,22496168,0,0,9020546


### 3.2 Exchange Rates

The local government funding values provided were stated in the local country's currency value at the time. I got data on worldwide exchange rates for the past years from the World Bank and applied it to the local government spending on malaria.

In [30]:
#Get exchange rate per country and apply it to Government column
Funding_exchange = pd.read_csv('Data/World_Bank/Exchange_Rates/Exchange_Rates.csv')
Funding_exchange = Funding_exchange.drop(['Country Code','Indicator Name','Indicator Code','Unnamed: 64'],axis=1)
Funding_exchange = Funding_exchange.rename(columns={'Country Name':'Country'})
years=[str(num) for num in range(2016,2019)]
Funding_exchange = pd.melt(Funding_exchange, id_vars='Country', value_vars=years,
                           var_name='Year', value_name='Exchange_Rate')
Funding_exchange['Year'] = pd.to_numeric(Funding_exchange['Year'])
Funding_df = pd.merge(Funding_df, Funding_exchange, how='left', left_on=['Country','Year'],
                      right_on=['Country','Year'])
Funding_df['Government (USD)'] = Funding_df['Government (local)']/Funding_df['Exchange_Rate']

In [31]:
#crosschecking and renaming country names
Funding_df.loc[Funding_df['Country']=='Democratic Republic of the Congo', ['Country']] = 'Democratic Republic of Congo'
old_names = ['Congo, Dem. Rep.','Congo, Rep.',"Cote d'Ivoire",'Gambia, The','Tanzania']
new_names = ['Democratic Republic of Congo','Congo',"Côte d'Ivoire",'Gambia',
             'United Republic of Tanzania']
Funding_exchange = Rename(old_names, new_names, Funding_exchange)
Names_Crosscheck(Funding_df['Country'],Funding_exchange['Country'], print_out=False)
#summing malaria Funding
column_list_funding = ['Global_Fund', 'PMI_USAID', 'World_Bank', 'UK', 'Government (USD)']
Funding_df['Total_Funding'] = Funding_df[column_list_funding].sum(axis=1)
#converting missing data to float
for i in Funding_df.index:
    if math.isnan(Funding_df.at[i,'Exchange_Rate'])==True:
        Funding_df.at[i,'Total_Funding'] = float('NaN')
#changing data type for 'Year' column
Funding_df['Year'] = pd.to_numeric(Funding_df['Year'])

In [32]:
#cross checking names and merging dataframes
Funding_inc = inc_cases_africa.drop('AU_Region',axis=1)
Names_Crosscheck(Funding_inc['Country'], Funding_df['Country'], print_out=False) 
Funding_years = [str(num) for num in range(2000,2019)]
Funding_inc = pd.melt(inc_cases_africa, id_vars='Country', value_vars=Funding_years,
                      var_name='Year', value_name='Inc_Cases')
Funding_inc['Year'] = pd.to_numeric(Funding_inc['Year'])
Funding_df = pd.merge(Funding_df, Funding_inc, how='inner', 
                      left_on=['Country','Year'], right_on=['Country','Year'])

Funding_df.head()

,Country,Year,Global_Fund,PMI_USAID,World_Bank,UK,Government (local),Exchange_Rate,Government (USD),Total_Funding,Inc_Cases
0,Algeria,2016.0,0.0,0,0,0,1743483,109.443067,15930.502069,1.593050e+04,0.00
1,Algeria,2017.0,0.0,0,0,0,1748498,110.973017,15756.064425,1.575606e+04,0.00
2,Algeria,2018.0,0.0,0,0,0,1812462,116.593792,15545.098706,1.554510e+04,0.00
3,Angola,2016.0,2725165.0,28133718,0,0,50874556,163.656434,310861.936316,3.116974e+07,219.99
4,Angola,2017.0,15453275.0,22496168,0,0,9020546,165.915951,54368.166306,3.800381e+07,228.91


### 3.3 Population

To be able to compare government spending between countries, I wanted to normalize the values to total funding per capita per year. I got data on population information by country and by year from the World Bank. Again, unwanted columns and rows were dropped, columns renamed, data types casted and country names crosschecked. The data frame was melted and the total funding per capita calculated. This information was then merged to the funding data.

In [33]:
#Adding population data 
Pop_df = pd.read_csv('Data/World_Bank/Population/Population.csv')
Pop_df = Pop_df.drop(['Country Code','Indicator Name','Indicator Code','Unnamed: 64'],axis=1)
Pop_years = [str(num) for num in range(2016,2019)]
Pop_df = pd.melt(Pop_df, id_vars='Country Name', value_vars=Pop_years,
                 var_name='Year', value_name='Population')
Names_Crosscheck(Funding_df['Country'], Pop_df['Country Name'], print_out=False)  
Pop_df = Rename(old_names, new_names, Pop_df)
Pop_df['Year'] = pd.to_numeric(Pop_df['Year'])
Funding_df = pd.merge(Funding_df, Pop_df, how='inner', 
                      left_on=['Country','Year'], right_on=['Country Name',"Year"])
Funding_df = Funding_df.drop('Country Name',axis=1)
Funding_df['Funding_per_capita'] = Funding_df['Total_Funding']/Funding_df['Population']

The number of malaria cases as a percentage of the entire population for each country and year was calculated and added to the data frame.

In [34]:
#Adding estimated cases to data frame
years=[str(num) for num in range(2016,2018)]
Funding_est = pd.melt(est_cases, id_vars='Country', value_vars=years,
                      var_name='Year', value_name='Est_Cases')
Funding_est['Year'] = pd.to_numeric(Funding_est['Year'])
Funding_df = pd.merge(Funding_df, Funding_est, how='left', left_on=['Country','Year'],
                      right_on=['Country','Year'])
#Adding estimated percentage of population with Malaria
Funding_df['Pop_Perc_Est_Cases'] = Funding_df['Est_Cases']/Funding_df['Population']

Funding_df.head(10)

,Country,Year,Global_Fund,PMI_USAID,World_Bank,UK,Government (local),Exchange_Rate,Government (USD),Total_Funding,Inc_Cases,Population,Funding_per_capita,Est_Cases,Pop_Perc_Est_Cases
0,Algeria,2016.0,0.0,0,0,0,1743483,109.443067,15930.502069,1.593050e+04,0.00,40551404.0,0.000393,0.0,0.000000
1,Algeria,2017.0,0.0,0,0,0,1748498,110.973017,15756.064425,1.575606e+04,0.00,41389198.0,0.000381,0.0,0.000000
2,Algeria,2018.0,0.0,0,0,0,1812462,116.593792,15545.098706,1.554510e+04,0.00,42228429.0,0.000368,NaN,NaN
3,Angola,2016.0,2725165.0,28133718,0,0,50874556,163.656434,310861.936316,3.116974e+07,219.99,28842484.0,1.080689,4485050.0,0.155502
4,Angola,2017.0,15453275.0,22496168,0,0,9020546,165.915951,54368.166306,3.800381e+07,228.91,29816748.0,1.274579,4615605.0,0.154799
5,Angola,2018.0,12123750.0,22000000,0,0,46457232,252.855748,183730.179823,3.430748e+07,228.91,30809762.0,1.113526,NaN,NaN
6,Benin,2016.0,2476172.0,17192827,0,0,17540458,592.605615,29598.872427,1.969860e+07,421.58,10872067.0,1.811854,4007210.0,0.368578
7,Benin,2017.0,25699563.0,16360849,0,0,4395380,580.656750,7569.670038,4.206798e+07,399.56,11175204.0,3.764404,4111699.0,0.367931
8,Benin,2018.0,4743095.0,16000000,0,0,611841,555.446458,1101.530113,2.074420e+07,386.18,11485048.0,1.806192,NaN,NaN
9,Botswana,2016.0,0.0,0,0,0,1310536,10.901158,120219.884890,1.202199e+05,0.81,2159944.0,0.055659,1150.0,0.000532


### 3.4 Insecticide Treated Nets (ITNs)

Insecticide Treated Nets are bed nets used at night to help prevent people being bitten by mosquitoes as they sleep. The percentage of the population with access to ITNs was added to the previous funding information.

In [35]:
#Adding percentage of population with ITN access to dataframe
Funding_ITN = pd.read_excel('Data/WHO/ITN.xls', sheet_name='ITN')
Funding_ITN_columns = Funding_ITN.columns.tolist()
Funding_ITN = Funding_ITN.rename(columns={'WHO region\nCountry/area':'Country',
                                          'Modelled percentage of population with access to an ITN':'Pop_Perc_ITN'})
#Fill empty cells with corresponding country
for i in Funding_ITN.index:
    if Funding_ITN.at[i,'Pop_Perc_ITN'] == '-':
        Funding_ITN = Funding_ITN.drop([i],axis=0)
Funding_ITN = Funding_ITN.reset_index(drop=True)
count=0
for cell in Funding_ITN['Country'].values:
    if count%3 == 0:
        country = cell
    if count%3 != 0:
        Funding_ITN.at[count, 'Country'] = country
    count += 1
Names_Crosscheck(Funding_ITN['Country'], Funding_df['Country'], print_out=False)
Funding_ITN = pd.merge(Funding_ITN, Funding_df, how='inner', left_on=['Country','Year'],
                       right_on=['Country','Year'])

Funding_ITN.head()

,Country,Year,Pop_Perc_ITN,Global_Fund,PMI_USAID,World_Bank,UK,Government (local),Exchange_Rate,Government (USD),Total_Funding,Inc_Cases,Population,Funding_per_capita,Est_Cases,Pop_Perc_Est_Cases
0,Angola,2016,20.9,2725165.0,28133718,0,0,50874556,163.656434,310861.936316,3.116974e+07,219.99,28842484.0,1.080689,4485050.0,0.155502
1,Angola,2017,31.8,15453275.0,22496168,0,0,9020546,165.915951,54368.166306,3.800381e+07,228.91,29816748.0,1.274579,4615605.0,0.154799
2,Angola,2018,37.5,12123750.0,22000000,0,0,46457232,252.855748,183730.179823,3.430748e+07,228.91,30809762.0,1.113526,NaN,NaN
3,Benin,2016,35.7,2476172.0,17192827,0,0,17540458,592.605615,29598.872427,1.969860e+07,421.58,10872067.0,1.811854,4007210.0,0.368578
4,Benin,2017,44.1,25699563.0,16360849,0,0,4395380,580.656750,7569.670038,4.206798e+07,399.56,11175204.0,3.764404,4111699.0,0.367931
